![Alt text](http://www.ucm.es/logo/ucm.png "a title")

<div align="center"> 
<font size=5> Máster en Nuevas Tecnologías Electrónicas y Fotónicas </font>
</div>
    
<div align="center"> 
<font size=4> Óptica Digital, curso 2021-2022 </font>
</div>

    
<div align="center"> 
<font size=4> Ejercicio A - Funcionamiento de los SLM </font>
</div>

- **Fecha**: 11 Dec 2022
        
- **Alumno**: Alex Recuenco (Alejandro Gonzalez Recuenco) 


# Introducción

En este ejercicio vamos a trabajar con la calibración del SLM para encontrar unas condiciones de modulación de amplitud y modulación de fase. Para ello, hay que diseñar una función de mérito para cada una de los dos tipos de modulación y encontrar una configuración de ángulos para cada una mediante una optimización.

## Cargar calibración del SLM

Hay que realizar estos pasos para cargar la calibración como un objeto *Jones_vector* de *pypol*.

In [ ]:
import numpy as np
from py_pol.jones_matrix import Jones_matrix
from py_pol.jones_vector import Jones_vector

In [ ]:
# Cargar datos del archivo NPZ (grabado con np.savez)
data = np.load("SLM_Jones_components.npz")
# Definir una lista con las componentes de la matriz de Jones: J00, J01, J10 y J11
components = [
    data["J0"] * np.exp(1j * data["d0"]),
    data["J1"] * np.exp(1j * data["d1"]),
    data["J2"] * np.exp(1j * data["d2"]),
    data["J3"] * np.exp(1j * data["d3"]),
]
# Crear el objeto py_pol
Jslm = Jones_matrix()
Jslm.from_components(components);

## Sistema

El sistema será igual que en el laboratorio: un haz inicial de polarización circular a derechas, un generador de estados (PSG) formado por un polarizador perfecto y una lámina de cuarto de onda que pueden rotar, el SLM, y un analizador de estados (PSA) formado por una lámina de cuarto de onda y un polarizador perfecto que pueden rotar.

# Análisis del SLM

En primer lugar, vamos a estudiar el SLM que tenemos. Para ello, en primer lugar vamos a representar su matriz de Jones empleando diversas combinaciones de parámetros.

## Parte real y parte imaginaria

Primero vamos a representar la parte real y la parte imaginaria de los elementos de matriz. Para ello, es necesario extraer las componentes con el método *parameters.components*.

In [ ]:
compontents = Jslm.parameters.components()
J00, J01, J10, J11 = components

Ahora, representa en una figura con 2 filas y 4 columnas la parte real y la parte imaginaria de las diferentes componentes en función de los niveles de gris.

In [ ]:
import matplotlib.pyplot as plt
from itertools import count

fig, ax = plt.subplots(nrows=4, ncols=2, sharex=True, sharey=True)
fig.suptitle("Componentes")
fig.supxlabel("gray level")
fig.supylabel("")

real_imag = map(lambda j: (np.real(j), np.imag(j)), components)
for index, (col1, col2), (real, imag) in zip(count(), ax, real_imag):
    col1.set_title(rf"$\Re(J_{{ {index:0>2b} }}) $")
    col1.plot(levels, real)
    col2.set_title(rf"$\Im(J_{{ {index:0>2b} }}) $")
    col2.plot(levels, imag)
fig.tight_layout()

plt.show()
plt.close()

## Modulo y fase

La representación en parte real e imaginaria, aunque correcta, no ofrece demasiada información. La representación en módulo y fase complejos ofrece algo más d información. Ahora, representa en una figura con 2 filas y 4 columnas el módulo y la fase complejas de las diferentes componentes en función de los niveles de gris.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=4, ncols=2, sharex=True)
fig.suptitle("Componentes")
fig.supxlabel("gray level")
fig.supylabel("")

mod_angle = map(lambda j: (np.abs(j), np.angle(j) / np.pi * 180), components)
for index, (col1, col2), (real, imag) in zip(count(), ax, mod_angle):
    col1.set_title(rf"$|J_{{ {index:0>2b} }}| $")
    col1.plot(levels, mod)
    col2.set_title(rf"$\phi(J_{{ {index:0>2b} }}) $")
    col2.plot(levels, angle)
fig.tight_layout()

plt.show()
plt.close()

## Descomposición polar

La representación módulo - fase ofrece información clara para algunos estados incidentes simples, pero para otros es más complicado. Otra manera de representar la información de la matriz de Jones es utilizando el teorema de descomposición polar. Este teorema permite descomponer las matrices de Jones en un polarizador y un retardador. Los 8 parámetros que emplearemos será la fase global de la matriz de Jones del SLM, la retardancia del retardador, el azimut y la elipticidad del autoestado rápido del retardador, la transmisión máxima y mínima del polarizador, y el azimut y la elipticidad del autoestado de transmisión del polarizador. 

Para ello, hay que empezar por dividir la matriz de Jones del SLM en la de un retardador y un polarizador. Para ello, se usa el método *analysis.decompose_pure*.

In [ ]:
Jr, Jd, parameters = Jslm.analysis.decompose_pure(all_info=True)
Jr: Jones_matrix
Jd: Jones_matrix
list(parameters.keys())

Ahora ya se pueden extraer los parámetros del retardador y del diatenuador. Para ello se utilizan los métodos *analysis.diattenuator* y *analysis.retarder*.

In [ ]:
# (trans_max, trans_min), (az_d, ellip_d) = Jd.analysis.diattenuator(transmissions="INTENSITY", angles='AZIMUTH')
# R, ang_r = Jr.analysis.retarder()
# # fase global de la matriz de Jones del SLM,
# # la retardancia del retardador,
# retardance=Jr.parameters.retardance()
# # el azimut y la elipticidad del autoestado rápido del retardador
# R, (az,ellip) = Jr.analysis.retarder(angles="AZIMUTH")
# # la transmisión máxima y mínima del polarizador,
# T_max, T_min = Jslm.parameters.transmissions()
# # y el azimut y la elipticidad del autoestado de transmisión del polarizador.
# Jslm.parameters.eigenvectors(angles="AZIMUTH")

list(parameters.keys())
global_phase = Jslm.parameters.global_phase()

characterization = {
    "global phase": global_phase,
    "retardance": parameters["R"],
    "azimuth R": parameters["azimuth R"],
    "ellipticity R": parameters["ellipticity R"],
    "Transmitance max": parameters["Tmax"],
    "Transmitance min": parameters["Tmin"],
    "azimuth D": parameters["azimuth D"],
    "ellipticity D": parameters["ellipticity D"],
}

characterization

Ya se pueden representar los parámetros en una figura de 2 filas y 4 columnas.

In [ ]:
import matplotlib.pyplot as plt
import itertools

fig, ax = plt.subplots(nrows=4, ncols=2, sharex=True)
ax_flatten = [row for col in ax for row in col]
fig.suptitle("Parametros")
fig.supxlabel("gray level")
fig.supylabel("")
ax_flatten

for axis, (title, dat) in zip(ax_flatten, characterization.items()):
    axis.set_title(title)
    axis.plot(levels, dat)
fig.tight_layout()

plt.show()
plt.close()

Por último, hacer una breve descripción del SLM en función de los parámetros obtenidos.

# Modulacion


In [ ]:
def transmission(angles, J: Jones_matrix) -> Jones_vector:
    (a1, a2, a3, a4) = angles
    Ein = Jones_vector().circular_light(
        intensity=2
    )  # intensity 2, since circular -> perfect diatenuator will remove half of it
    Jp1 = Jones_matrix().diattenuator_perfect(azimuth=a1)
    Jr1 = Jones_matrix().quarter_waveplate(azimuth=a2)
    Jr2 = Jones_matrix().quarter_waveplate(azimuth=a2)
    Jp2 = Jones_matrix().diattenuator_perfect(azimuth=a2)
    # Efin: Jones_vector = Jp2 * Jr2 * Jslm * Jr1 * Ein
    # Ifin, phase = Efin.parameters.intensity(), Efin.parameters.global_phase()
    # global_ = Efin.parameters.inte.global_phase()
    Efin: Jones_vector = Jp2 * Jr2 * J * Jr1 * Jp1 * Ein
    return Efin


def parameters_amplitud_phase(E: Jones_vector):
    global_phase_init = E.parameters.global_phase()
    global_phase = global_phase
    return E.parameters.intensity(), (E.parameters.global_phase() % 2 * np.pi)


def parameters_az_el(E: Jones_vector):
    return E.parameters.azimuth_ellipticity()

# Modulación de amplitud

## Función de mérito

Define una buena función de mérito que sirva para encontrar una configuración de modulación de amplitud.

In [ ]:
ideal_amplitud = np.arange(256) / 256
ideal_phase = np.zeros(ideal_amplitud.shape) + np.pi


def plot_amplitud_phase(t, phase, *, levels=levels):
    fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2)

    fig.suptitle("Amplitud Calibration")
    ax1.set_title("Amplitud")
    ax1.plot(levels, t)
    ax2.set_title("Phase")
    ax2.plot(levels, phase)


def err(
    angles,
    *,
    J: Jones_matrix = Jslm,
    ideal_amplitud=ideal_amplitud,
    ideal_phase=ideal_phase
):
    E = transmission(angles, J)
    t, phase = parameters_amplitud_phase(E)
    normalized_phase = phase
    normalized_ideal_phase = ideal_phase
    phase_diff = normalized_phase - normalized_ideal_phase
    phase_err = np.abs(phase_diff)
    t_err = np.abs((t - ideal_amplitud))
    return np.sum(t_err) + np.sum(phase_err)


plot_amplitud_phase(ideal_amplitud, ideal_phase)

## Optimización

Encuentra la configuración de modulación de amplitud empleando la función de mérito definida en el apartado anterior. El resultado debe ser los ángulos de los polarizadores y láminas de cuarto de onda del PSG y del PSA, además de una representación de la transmisión en amplitud y fase del sistema en dicha configuración.

In [ ]:
from scipy.optimize import minimize

solution = minimize(
    err,
    np.random.uniform(np.pi / 4, np.pi / 2, size=4),
    options={"disp": True},
    method="Nelder-Mead",
    bounds=[[0, 2 * np.pi]] * 4,
)
solution

In [ ]:
print(
    f"Solution is {solution['x']}. Angles are, in order: Polarizer, Quarter waveplet, quarter Waveplet Polarizer"
)

In [ ]:
E = transmission(solution["x"], J=Jslm)

t, phase = parameters_amplitud_phase(E)
plot_amplitud_phase(t, phase)

La modulación de la fase sale bastante mal, da igual como lo intente para mantenerla constante.

# Modulación de fase

## Función de mérito

Define una buena función de mérito que sirva para encontrar una configuración de modulación de fase.

In [ ]:
ideal_phase = np.pi * 2 * np.arange(256) / 255
ideal_amplitud = np.ones(ideal_amplitud.shape)

fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, sharex=True)
fig.suptitle("Ideal")
ax1.set_title("Amplitud")
ax2.set_title("Phase")
ax1.plot(levels, ideal_amplitud)
ax2.plot(levels, ideal_phase)

## Optimización

Encuentra la configuración de modulación de fase empleando la función de mérito definida en el apartado anterior. El resultado debe ser los ángulos de los polarizadores y láminas de cuarto de onda del PSG y del PSA, además de una representación de la transmisión en amplitud y fase del sistema en dicha configuración.

In [ ]:
from functools import partial
from scipy.optimize import minimize


def err(
    angles,
    *,
    J: Jones_matrix = Jslm,
    ideal_amplitud=ideal_amplitud,
    ideal_phase=ideal_phase,
):
    E = transmission(angles, J)
    t, phase = parameters_amplitud_phase(E)
    normalized_phase = phase / (2 * np.pi)
    normalized_ideal_phase = ideal_phase / (2 * np.pi)
    phase_diff = normalized_phase - normalized_ideal_phase
    phase_err = np.sum((phase_diff) ** 2)
    t_err = np.sum((t - ideal_amplitud) ** 2)
    return phase_err * t_err / (phase_err + t_err)


ideal_phase = np.pi * 2 * np.arange(256) / 255
ideal_amplitud = np.ones(ideal_amplitud.shape)

phase_err = partial(err, ideal_phase=ideal_phase, ideal_amplitud=ideal_amplitud)

solution = minimize(
    err,
    np.random.uniform(np.pi / 4, 3 * np.pi / 2, size=4),
    options={"disp": True},
    method="Nelder-Mead",
    bounds=[[0, 2 * np.pi]] * 4,
)


print(
    f"Solution is {solution}. Angles are, in order: Polarizer, Quarter waveplet, quarter Waveplet Polarizer"
)
solution

In [ ]:
E = transmission(solution["x"], J=Jslm)

t, phase = parameters_amplitud_phase(E)
plot_amplitud_phase(t, phase)

Lo mas seguro es que lo estoy haciendo mal.

Sin embargo, cuando veo las graficas de la fase global, parece que es muy dificil consegui un cambio de esta fase global en nuestro sistema.